In [1]:
import absbox 

#Hastructure version ✅Connected, local lib:0.27.24, server:0.27.6

In [2]:
loan_1 = ['Loan', 
  {'originBalance': 34234.57286845755,
    'originRate': ['fix', 0.19082680623970882], 
    'originTerm': 1, 
    'freq': 'Monthly', 
    'type': 'i_p', 
    'originDate': '2024-01-27'}, 
    {'currentBalance': 19540.658974405575, 
     'currentRate': 0.19082680623970882, 
     'remainTerm': 1, 
     'status': 'current'}]

loan_2 = ['Loan', 
    {'originBalance': 107725.2358849491, 
     'originRate': ['fix', 0.1588154059357318], 
     'originTerm': 2,
    'freq': 'Monthly',
      'type': 'i_p',
      'originDate': '2024-01-27'},
    {'currentBalance': 51748.57962720552, 
     'currentRate': 0.1588154059357318, 
     'remainTerm': 1, 'status': 'current'}]

loan_3 = ['Loan', {'originBalance': 68958.38006061042,
         'originRate': ['fix', 0.15866267897996908], 
           'originTerm': 3, 
        'freq': 'Monthly',
          'type': 'i_p', 
          'originDate': '2024-01-27'}, 
          {'currentBalance': 37027.61253184687, 
           'currentRate': 0.15866267897996908, 
           'remainTerm': 2, 
           'status': 'current'}]
    
loan_4 = ['Loan', 
     {'originBalance': 89056.6957926069, 
    'originRate': ['fix', 0.12683991247737755], 
    'originTerm': 4, 
    'freq': 'Monthly', 
    'type': 'i_p', 
    'originDate': '2024-01-27'}, 
    {'currentBalance': 46509.47402266063, 
     'currentRate': 0.12683991247737755, 
     'remainTerm': 3, 
     'status': 'current'}]

loan_5 = ['Loan',
      {'originBalance': 57523.59506021201, 
       'originRate': ['fix', 0.1306138244082632], 
       'originTerm': 5, 
       'freq': 'Monthly', 
       'type': 'i_p', 
       'originDate': '2024-01-27'}, 
       {'currentBalance': 45337.9972060108, 
        'currentRate': 0.1306138244082632, 
        'remainTerm': 2,
        'status': 'current'}]
 
loan_6 = ['Loan', 
         {'originBalance': 398.32467231794726, 
          'originRate': ['fix', 0.12036556280728988], 
          'originTerm': 6, 
          'freq': 'Monthly', 
          'type': 'i_p', 
          'originDate': '2024-01-27'}, 
          {'currentBalance': 314.6644277486739, 
           'currentRate': 0.12036556280728988, 
           'remainTerm': 3,
             'status': 'current'}]

In [3]:
pool = {'name':'name', 'assets':[loan_1,loan_2,loan_3,loan_4,loan_5,loan_6],'issuanceStat':{"IssuanceBalance":300000}}

dates={"collect": ["2024-02-29","2024-03-31"]
        ,"pay": ["2024-02-15"  
                      ,"2024-03-15"]  
        ,"stated":"2027-11-04"
        ,"poolFreq":"MonthFirst"
        ,"payFreq":["DayOfMonth",15]
        }

accounts={"collections":{"balance":50_000_000},
            "revolBuyAcc":{"balance":100_000}
            ,"purchaserAccount": {"balance":0}
            ,"sellerAccount": {"balance": 0}
          ,"cashReserve":{"balance":0}}
          
bonds={"A1":{"balance":150_000_000
                    ,"rate":0.15
                    ,"originBalance":200_000_000
                    ,"originRate":0.0858
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"fix": 0.15, "dayCount":"DC_ACT_365A"}
                    ,"maturityDate":"2027-11-04"
                    ,"bondType":{"Lockout":"2023-11-04"}}
                    ,"B":{
                    "balance":1
                    ,"rate":0.0
                    ,"originBalance":1000
                    ,"originRate":0.0
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}
                    }
                    }

fees={"technology_fee":{"type":{"annualPctFee":[("bondBalance",),0.004]}}}

waterfall = {"default": [
              ["payFee","collections",["technology_fee"]]
              ,["accrueAndPayInt","collections",["A1"]]
              ,["transfer" ,"collections","revolBuyAcc"]
              ,["buyAsset",["Current|Defaulted",1.0,0],"revolBuyAcc",None] #This line is the problematic one. If I remove it the server will run the deal
              ,["payIntResidual","collections","B"]]
  
  ,"endOfCollection":[["calcFee","technology_fee"] ,["calcInt","A1"]]

        ,"cleanUp":[["sellAsset", ["Current|Defaulted",1.0,0], "collections"] #I also comment out this line as well
                    ,["transfer" ,"cashReserve",'collections']
                    ,["transfer" ,"revolBuyAcc",'collections']
                    ,["accrueAndPayInt","collections",["A1"]]
                    ,["payPrin","collections",["A1"]]
                    ,["payIntResidual","collections","B"]]}
        
collects=[["CollectedPrincipal","collections"]
              ,["CollectedRecoveries","collections"]
              ,["CollectedPrepayment","collections"]
              ,["CollectedInterest","collections"]
              ]

trigger=None
deal_data = {
          "name":"Deal"
          ,"dates":dates
          ,"pool":pool
          ,"accounts":accounts
          ,"fees":fees
          ,"bonds":bonds
          ,"waterfall":waterfall
          ,"collect":collects
          ,"trigger":trigger
          ,"status":"Revolving"
        } 


deal=absbox.mkDeal(deal_data)


revol_assets = [loan_1,
                loan_2,
                loan_3,
                loan_4,
                loan_5,
                loan_6]

revolving=("revolving"
        ,["constant",*revol_assets]
        ,("Pool",("Loan",{"CDR":0.005},{"CPR":0.1},None,None)
                                            ,None
                                            ,None))
      
localAPI = absbox.API("https://absbox.org/api/dev",lang='english',check=False)
#localAPI = absbox.API("http://localhost:8081",lang='english',check=False)

r = localAPI.run(deal
              ,poolAssump = ("Pool",("Loan",{"CDR":0.005},{"CPR":0.1},None,None)
                                            ,None
                                            ,None)
              ,runAssump = [("inspect"
                                ,("MonthEnd",("status",("Amortizing")))
                                ,("MonthEnd",("accountBalance",))
                                ,("MonthEnd",("poolBalance",))
                                ,("MonthEnd",("bondBalance",))
                                )
                            ,revolving,
                            ("report",
                                {"dates":["OffsetDateDattern",["DayOfMonth",1],14]}),("call",{"afterDate":"2025-11-04"})
                            #,("stop","2024-05-30")
                          ]
            ,read=True)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.4, server:0.28.1

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
No waterfall distribution found on date2024-03-15with statusRevolving
No waterfall distribution found on date2024-04-15with statusRevolving
No waterfall distribution found on date2024-05-15with statusRevolving
No waterfall distribution found on date2024-06-15with statusRevolving
No waterfall distribution found on date2024-07-15with statusRevolving
No waterfall distribution found on date2024-08-15with statusRevolving
No waterfall distribution found on date2024-09-15with statusRevolving
No waterfall distribution found on date2024-10-15with statusRevolving
No waterfall distribution found on date2024-11-15with statusRevolving
No waterfall distribution found on date2024-12-15with statusRevolving
No waterfall distribution found on date2025-01-15with statusRevolving
No waterfall distribution found on date2025-02-15with statusRevolving
No waterfall distribution found on date2025-03-15with statusRevolving
No waterfall distribution found on date2025-04-15with statusRevolving
No waterfall distribution found on date2025-05-15with statusRevolving
No waterfall distribution found on date2025-06-15with statusRevolving
No waterfall distribution found on date2025-07-15with statusRevolving
No waterfall distribution found on date2025-08-15with statusRevolving
No waterfall distribution found on date2025-09-15with statusRevolving
No waterfall distribution found on date2025-10-15with statusRevolving
No waterfall distribution found on date2025-11-15with statusRevolving

In [7]:
r['result']['report']['balanceSheet']

asset                                                         \
           cashReserve collections purchaserAccount revolBuyAcc sellerAccount   
reportDate                                                                      
2024-03-14         0.0  50000000.0              0.0    100000.0           0.0   
2024-03-15         0.0         0.0              0.0         0.0           0.0   
2024-04-15         0.0         0.0              0.0         0.0           0.0   
2024-05-15         0.0         0.0              0.0         0.0           0.0   
2024-06-15         0.0         0.0              0.0         0.0           0.0   
2024-07-15         0.0         0.0              0.0         0.0           0.0   
2024-08-15         0.0         0.0              0.0         0.0           0.0   
2024-09-15         0.0         0.0              0.0         0.0           0.0   
2024-10-15         0.0         0.0              0.0         0.0           0.0   
2024-11-15         0.0         0.0              0.0         0.0           0.0   
2024-12-15         0.0         0.0              0.0         0.0           0.0   
2025-01-15         0.0         0.0              0.0         0.0           0.0   
2025-02-15         0.0         0.0              0.0         0.0           0.0   
2025-03-15         0.0         0.0              0.0         0.0           0.0   
2025-04-15         0.0         0.0              0.0         0.0           0.0   
2025-05-15         0.0         0.0              0.0         0.0           0.0   
2025-06-15         0.0         0.0              0.0         0.0           0.0   
2025-07-15         0.0         0.0              0.0         0.0           0.0   
2025-08-15         0.0         0.0              0.0         0.0           0.0   
2025-09-15         0.0         0.0              0.0         0.0           0.0   
2025-10-15         0.0         0.0              0.0         0.0           0.0   
2025-11-15         0.0         0.0              0.0         0.0           0.0   

                                                            liability       \
           Pool Performing Pool Defaulted Pool Recovery            A1    B   
reportDate                                                                   
2024-03-14       300000.00           0.00           0.0  1.500000e+08  1.0   
2024-03-15       300000.00           0.00           0.0  1.500000e+08  1.0   
2024-04-15     48445673.47          53.85           0.0  1.500000e+08  1.0   
2024-05-15     30936693.26       20654.42           0.0  1.500000e+08  1.0   
2024-06-15     25348229.29       33397.36           0.0  1.500000e+08  1.0   
2024-07-15     27258752.22       44186.32           0.0  1.500000e+08  1.0   
2024-08-15     29517669.24       55414.28           0.0  1.500000e+08  1.0   
2024-09-15     24249966.57       67977.89           0.0  1.500000e+08  1.0   
2024-10-15     24577316.25       78299.39           0.0  1.500000e+08  1.0   
2024-11-15     24049856.85       88422.84           0.0  1.500000e+08  1.0   
2024-12-15     22466706.84       98659.16           0.0  1.500000e+08  1.0   
2025-01-15     21231477.80      107913.24           0.0  1.500000e+08  1.0   
2025-02-15     20449789.44      116949.97           0.0  1.500000e+08  1.0   
2025-03-15     19187575.92      125654.00           0.0  1.500000e+08  1.0   
2025-04-15     18203082.92      133030.60           0.0  1.500000e+08  1.0   
2025-05-15     17018015.95      140778.35           0.0  1.500000e+08  1.0   
2025-06-15     15943043.81      147788.11           0.0  1.500000e+08  1.0   
2025-07-15     14731730.04      154573.89           0.0  1.500000e+08  1.0   
2025-08-15     13656637.19      160641.89           0.0  1.500000e+08  1.0   
2025-09-15     12438782.41      166454.53           0.0  1.500000e+08  1.0   
2025-10-15     11262236.21      171748.81           0.0  1.500000e+08  1.0   
2025-11-15     10132474.72      176387.72           0.0  1.398675e+08  1.0   

                                        

In [6]:
r['result']['report']['balanceSheet']

asset                                                         \
           cashReserve collections purchaserAccount revolBuyAcc sellerAccount   
reportDate                                                                      
2024-03-14         0.0  50000000.0              0.0    100000.0           0.0   
2024-03-15         0.0         0.0              0.0         0.0           0.0   
2024-04-15         0.0         0.0              0.0         0.0           0.0   
2024-05-15         0.0         0.0              0.0         0.0           0.0   
2024-06-15         0.0         0.0              0.0         0.0           0.0   
2024-07-15         0.0         0.0              0.0         0.0           0.0   
2024-08-15         0.0         0.0              0.0         0.0           0.0   
2024-09-15         0.0         0.0              0.0         0.0           0.0   
2024-10-15         0.0         0.0              0.0         0.0           0.0   
2024-11-15         0.0         0.0              0.0         0.0           0.0   
2024-12-15         0.0         0.0              0.0         0.0           0.0   
2025-01-15         0.0         0.0              0.0         0.0           0.0   
2025-02-15         0.0         0.0              0.0         0.0           0.0   
2025-03-15         0.0         0.0              0.0         0.0           0.0   
2025-04-15         0.0         0.0              0.0         0.0           0.0   
2025-05-15         0.0         0.0              0.0         0.0           0.0   
2025-06-15         0.0         0.0              0.0         0.0           0.0   
2025-07-15         0.0         0.0              0.0         0.0           0.0   
2025-08-15         0.0         0.0              0.0         0.0           0.0   
2025-09-15         0.0         0.0              0.0         0.0           0.0   
2025-10-15         0.0         0.0              0.0         0.0           0.0   
2025-11-15         0.0         0.0              0.0         0.0           0.0   

                                                           liability       \
           Pool Performing Pool Defaulted Pool Recovery           A1    B   
reportDate                                                                  
2024-03-14       300000.00           0.00           0.0  150000000.0  1.0   
2024-03-15       300000.00           0.00           0.0  150000000.0  1.0   
2024-04-15     48445673.47          53.85           0.0  150000000.0  1.0   
2024-05-15     30936693.26       20654.42           0.0  150000000.0  1.0   
2024-06-15     25384052.52       33397.36           0.0  150000000.0  1.0   
2024-07-15     27287650.80       44201.58           0.0  150000000.0  1.0   
2024-08-15     29548792.96       55441.45           0.0  150000000.0  1.0   
2024-09-15     24286696.63       68018.29           0.0  150000000.0  1.0   
2024-10-15     24620442.52       78355.42           0.0  150000000.0  1.0   
2024-11-15     24094138.15       88496.64           0.0  150000000.0  1.0   
2024-12-15     22516330.99       98751.81           0.0  150000000.0  1.0   
2025-01-15     21285300.67      108026.35           0.0  150000000.0  1.0   
2025-02-15     20507757.48      117085.99           0.0  150000000.0  1.0   
2025-03-15     19249762.97      125814.68           0.0  150000000.0  1.0   
2025-04-15     18268857.43      133215.20           0.0  150000000.0  1.0   
2025-05-15     17088338.37      140990.94           0.0  150000000.0  1.0   
2025-06-15     16017550.85      148029.65           0.0  150000000.0  1.0   
2025-07-15     14810770.88      154847.16           0.0  150000000.0  1.0   
2025-08-15     13739796.95      160947.73           0.0  150000000.0  1.0   
2025-09-15     12526605.49      166795.77           0.0  150000000.0  1.0   
2025-10-15     11354621.45      172127.43           0.0  150000000.0  1.0   
2025-11-15     10229121.60      176804.39           0.0  139770878.4  1.0   

                                                                 

In [4]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-03-31,48445673.47,51296.76,1574.57,1127.30,53.85,0,53.85,0.142274,5.129676e+04,1127.30,0,53.85,0,53.85
2024-04-01,48445673.47,0.00,0.00,0.00,0.00,0,0.00,0.135931,5.129676e+04,1127.30,0,53.85,0,53.85
2024-05-01,30936693.26,17057391.17,602861.31,430988.47,20600.57,0,20600.57,0.142885,1.710869e+07,432115.77,0,20654.42,0,20654.42
2024-06-01,25384052.52,19533293.89,345822.71,266638.05,12742.94,0,12742.94,0.168708,3.664198e+07,698753.82,0,33397.36,0,33397.36
2024-07-01,27287650.80,16043396.56,296325.11,226037.77,10804.22,0,10804.22,0.145923,5.268538e+07,924791.59,0,44201.58,0,44201.58
2024-08-01,29548792.96,12159559.73,321056.75,235187.54,11239.87,0,11239.87,0.141198,6.484494e+07,1159979.13,0,55441.45,0,55441.45
2024-09-01,24286696.63,15740282.08,351702.65,263123.61,12576.84,0,12576.84,0.137663,8.058522e+07,1423102.74,0,68018.29,0,68018.29
2024-10-01,24620442.52,13832841.39,286584.45,216266.11,10337.13,0,10337.13,0.133413,9.441806e+07,1639368.85,0,78355.42,0,78355.42
2024-11-01,24094138.15,12741025.64,285039.77,212199.32,10141.22,0,10141.22,0.144149,1.071591e+08,1851568.17,0,88496.64,0,88496.64


In [5]:
30936693.26 - 19533293.89 - 266638.05 - 12742.94

11124018.38

In [7]:
25348229.29 - 25384052.52

-35823.23000000045

In [8]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-03-31,48445673.47,51296.76,1574.57,1127.30,53.85,0,53.85,0.142274,5.129676e+04,1127.30,0,53.85,0,53.85
2024-04-01,48445673.47,0.00,0.00,0.00,0.00,0,0.00,0.135931,5.129676e+04,1127.30,0,53.85,0,53.85
2024-05-01,30936693.26,17057391.17,602861.31,430988.47,20600.57,0,20600.57,0.142885,1.710869e+07,432115.77,0,20654.42,0,20654.42
2024-06-01,25348229.29,19533293.89,345822.71,266638.05,12742.94,0,12742.94,0.168717,3.664198e+07,698753.82,0,33397.36,0,33397.36
2024-07-01,27258752.22,16030776.88,295878.87,225718.78,10788.96,0,10788.96,0.145927,5.267276e+07,924472.60,0,44186.32,0,44186.32
2024-08-01,29517669.24,12142986.33,320728.37,234938.48,11227.96,0,11227.96,0.141197,6.481575e+07,1159411.08,0,55414.28,0,55414.28
2024-09-01,24249966.57,15722998.69,351333.14,262846.45,12563.61,0,12563.61,0.137660,8.053874e+07,1422257.53,0,67977.89,0,67977.89
2024-10-01,24577316.25,13815620.95,286140.42,215939.05,10321.50,0,10321.50,0.133418,9.435436e+07,1638196.58,0,78299.39,0,78299.39
2024-11-01,24049856.85,12718903.97,284540.27,211827.64,10123.45,0,10123.45,0.144151,1.070733e+08,1850024.22,0,88422.84,0,88422.84


In [6]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-03-31,48445673.47,51296.76,1574.57,1127.30,53.85,0,53.85,0.142274,5.129676e+04,1127.30,0,53.85,0,53.85
2024-04-01,48445673.47,0.00,0.00,0.00,0.00,0,0.00,0.135931,5.129676e+04,1127.30,0,53.85,0,53.85
2024-05-01,30936693.26,17057391.17,602861.31,430988.47,20600.57,0,20600.57,0.142885,1.710869e+07,432115.77,0,20654.42,0,20654.42
2024-06-01,25348229.29,19533293.89,345822.71,266638.05,12742.94,0,12742.94,0.168717,3.664198e+07,698753.82,0,33397.36,0,33397.36
2024-07-01,27258752.22,16030776.88,295878.87,225718.78,10788.96,0,10788.96,0.145927,5.267276e+07,924472.60,0,44186.32,0,44186.32
2024-08-01,29517669.24,12142986.33,320728.37,234938.48,11227.96,0,11227.96,0.141197,6.481575e+07,1159411.08,0,55414.28,0,55414.28
2024-09-01,24249966.57,15722998.69,351333.14,262846.45,12563.61,0,12563.61,0.137660,8.053874e+07,1422257.53,0,67977.89,0,67977.89
2024-10-01,24577316.25,13815620.95,286140.42,215939.05,10321.50,0,10321.50,0.133418,9.435436e+07,1638196.58,0,78299.39,0,78299.39
2024-11-01,24049856.85,12718903.97,284540.27,211827.64,10123.45,0,10123.45,0.144151,1.070733e+08,1850024.22,0,88422.84,0,88422.84


In [7]:
r['accounts']['revolBuyAcc']

,balance,change,memo
date,,,
2024-03-15,48317213.12,48217213.12,"<Transfer:collections,revolBuyAcc>"
2024-03-15,0.00,-48317213.12,<PurchaseAsset:>
2024-04-15,0.00,0.00,"<Transfer:collections,revolBuyAcc>"
2024-04-15,0.00,0.00,<PurchaseAsset:>
2024-05-15,14224210.94,14224210.94,"<Transfer:collections,revolBuyAcc>"
2024-05-15,0.00,-14224210.94,<PurchaseAsset:>
2024-06-15,18177807.58,18177807.58,"<Transfer:collections,revolBuyAcc>"
2024-06-15,0.00,-18177807.58,<PurchaseAsset:>
2024-07-15,14648069.81,14648069.81,"<Transfer:collections,revolBuyAcc>"


In [13]:
import pandas
pandas.set_option('display.max_rows', None)


In [14]:
r['accounts']['collections']

,balance,change,memo
date,,,
2024-03-15,5.000000e+07,0.000000e+00,<SeqPayFee:technology_fee>
2024-03-15,4.821721e+07,-1.782787e+06,<PayInt:A1>
2024-03-15,0.000000e+00,-4.821721e+07,"<Transfer:collections,revolBuyAcc>"
2024-03-15,0.000000e+00,0.000000e+00,<PayYield:B>
2024-03-31,1.574570e+03,1.574570e+03,<Pool:CollectedInterest>
2024-03-31,2.701870e+03,1.127300e+03,<Pool:CollectedPrepayment>
2024-03-31,2.701870e+03,0.000000e+00,<Pool:CollectedRecoveries>
2024-03-31,5.399863e+04,5.129676e+04,<Pool:CollectedPrincipal>
2024-04-01,5.399863e+04,0.000000e+00,<Pool:CollectedInterest>


In [8]:
r['result']['status']

,Date,From,To
0,2025-11-15,Revolving,Called
1,2025-11-15,DealEnd,Clean Up


In [16]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-03-31,1.584063e+07,51296.76,1574.57,1127.30,53.85,0,53.85,0.142281,5.129676e+04,1127.30,0,53.85,0,53.85
2024-04-01,1.584063e+07,0.00,0.00,0.00,0.00,0,0.00,0.135931,5.129676e+04,1127.30,0,53.85,0,53.85
2024-05-01,-1.668354e+06,17057391.17,602861.31,430988.47,20600.57,0,20600.57,0.122182,1.710869e+07,432115.77,0,20654.42,0,20654.42
2024-06-01,-2.009460e+07,19533293.89,345822.71,266638.05,12742.94,0,12742.94,0.130806,3.664198e+07,698753.82,0,33397.36,0,33397.36
2024-07-01,-2.232349e+07,16030776.88,295878.87,225718.78,10788.96,0,10788.96,0.140478,5.267276e+07,924472.60,0,44186.32,0,44186.32
2024-08-01,-2.994927e+07,12142986.33,320728.37,234938.48,11227.96,0,11227.96,0.136319,6.481575e+07,1159411.08,0,55414.28,0,55414.28
2024-09-01,-3.766054e+07,15722998.69,351333.14,262846.45,12563.61,0,12563.61,0.128937,8.053874e+07,1422257.53,0,67977.89,0,67977.89
2024-10-01,-4.060533e+07,13815620.95,286140.42,215939.05,10321.50,0,10321.50,0.134671,9.435436e+07,1638196.58,0,78299.39,0,78299.39
2024-11-01,-5.233625e+07,12718903.97,284540.27,211827.64,10123.45,0,10123.45,0.143246,1.070733e+08,1850024.22,0,88422.84,0,88422.84
